In [2]:
print('hola')

hola



  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB 1.3 MB/s eta 0:00:09
    --------------------------------------- 0.2/11.5 MB 2.9 MB/s eta 0:00:04
   - -------------------------------------- 0.3/11.5 MB 3.0 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/11.5 MB 3.8 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.5 MB 4.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.4/11.5 MB 5.3 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/11.5 MB 6.1 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.5 MB 6.5 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.5 MB 7.0 MB/s eta 0:00:02
   ------------ ----------------

In [15]:
import pandas as pd

# Load Excel file into a DataFrame
sheet_name = 'Sheet1'  # Replace with the actual sheet name if needed
CHS = pd.read_excel('CHS_files.xlsx', sheet_name=sheet_name)
CHE = pd.read_excel('CHE_files.xlsx', sheet_name=sheet_name)
# Specify the column to check for empty records
column_to_check = 'abstract'  # Replace with the actual column name

# Check how many records are empty in the specified column
empty_records_count_che = CHE[column_to_check].isna().sum()
empty_records_count_chs = CHS[column_to_check].isna().sum()

print(f'The number of empty records in the column "{column_to_check}" for CHE file is: {empty_records_count_che} out of {CHE.Label.count()}')
print(f'The number of empty records in the column "{column_to_check}" for CHS file is: {empty_records_count_chs} out of {CHS.Label.count()}')


The number of empty records in the column "abstract" for CHE file is: 376 out of 9788
The number of empty records in the column "abstract" for CHS file is: 1036 out of 10153


In [24]:
non_abstract_CHS = CHS[CHS.abstract.isna()].reset_index()

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [49]:
import requests
from bs4 import BeautifulSoup
import re

def search_paper_abstract(paper_title):
    # Replace spaces with '+' for the search query
    query = paper_title.replace(' ', '+')
    url = f'https://scholar.google.com/scholar?q={query}'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    # Make a request to Google Scholar
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch results for {paper_title}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the first result link
    result = soup.find('h3', {'class': 'gs_rt'})
    if result:
        paper_link = result.a['href']
        print(f"Paper found: {paper_link}")
        
        # Make a request to the paper's page
        response = requests.get(paper_link, headers=headers)
        if response.status_code != 200:
            print(f"Failed to fetch the paper page for {paper_title}")
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract the abstract (the method may vary depending on the website structure)
        abstract = None
        if "arxiv.org" in paper_link:
            abstract = soup.find('blockquote', {'class': 'abstract mathjax'}).text
        elif "springer.com" in paper_link:
            abstract = soup.find('section', {'class': 'Abstract'}).text
        # Add more conditions here for other sources
        
        if abstract:
            return abstract.strip()
        else:
            print(f"Abstract not found for {paper_title}")
            return None
    else:
        print(f"No results found for {paper_title}")
        return None

# List of paper titles
paper_titles = non_abstract_CHS['short_title'].to_list()
paper_titles = prueba

for title in paper_titles:
    print(f"Searching for: {title}")
    abstract = search_paper_abstract(title)
    if abstract:
        print(f"Abstract for '{title}':\n{abstract}\n")
    else:
        print(f"Could not retrieve abstract for '{title}'\n")


Searching for: Short term respiratory effects of acute exposure to chlorine due to a swimming pool accident
Failed to fetch results for Short term respiratory effects of acute exposure to chlorine due to a swimming pool accident
Could not retrieve abstract for 'Short term respiratory effects of acute exposure to chlorine due to a swimming pool accident'

Searching for: Wheezing in the workplace: Occupational asthma is on the rise
Failed to fetch results for Wheezing in the workplace: Occupational asthma is on the rise
Could not retrieve abstract for 'Wheezing in the workplace: Occupational asthma is on the rise'

Searching for: Hazard data bank: chlorine
Failed to fetch results for Hazard data bank: chlorine
Could not retrieve abstract for 'Hazard data bank: chlorine'

Searching for: Chemical safe practice pamphlet: chlorine
Failed to fetch results for Chemical safe practice pamphlet: chlorine
Could not retrieve abstract for 'Chemical safe practice pamphlet: chlorine'

Searching for: C

In [52]:
testing = CHS[CHS.abstract.notna()].head()
prueba = testing['short_title'].to_list()
testing

,short_title,abstract,Label,Subgroup,url,PMID,PMCID,Match
0,Short term respiratory effects of acute exposu...,Acute exposure to Cl2 causes lung damage and r...,Relevant,Clinical,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,11351056,PMC1740147,NaN
1,Wheezing in the workplace: Occupational asthma...,It took almost a year for George Stephens to l...,Relevant,Clinical,https://www.sciencedirect.com/science/article/...,NaN,NaN,NaN
2,Hazard data bank: chlorine,This data sheet covers: description; uses; fir...,Relevant,Clinical,Internal Retrieval,NaN,NaN,NaN
3,Chemical safe practice pamphlet: chlorine,Practical information is provided under the he...,Relevant,Clinical,Internal Retrieval,NaN,NaN,NaN
4,Chlorine gas release associated with employee ...,"On June 27, 2011, a worker at a poultry proces...",Relevant,Clinical,Internal Retrieval,NaN,NaN,NaN


In [56]:
from scholarly import scholarly, ProxyGenerator

def search_paper_abstract(paper_title):
    pg = ProxyGenerator()
    pg.FreeProxies()  # Optionally use proxies if necessary
    scholarly.use_proxy(pg)
    
    search_query = scholarly.search_pubs(paper_title)
    try:
        paper = next(search_query)
        return paper['bib']['abstract']
    except StopIteration:
        print(f"No results found for '{paper_title}'")
        return None
    except KeyError:
        print(f"Abstract not found for '{paper_title}'")
        return None



for title in paper_titles:
    print(f"Searching for: {title}")
    abstract = search_paper_abstract(title)
    if abstract:
         print(f"Abstract for '{title}':\n{abstract}\n")
    else:
        print(f"Could not retrieve abstract for '{title}'\n")

Searching for: Short term respiratory effects of acute exposure to chlorine due to a swimming pool accident


c:\Users\facua\anaconda3\envs\chlorine\Lib\site-packages\scholarly\_scholarly.py:312: SyntaxWarning: invalid escape sequence '\d'
  m = re.search("cites=[\d+,]*", object["citedby_url"])


MaxTriesExceededException: Cannot Fetch from Google Scholar.

In [2]:
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer

device = torch.device("cpu")

tokenizer = GPT2Tokenizer.from_pretrained("stanford-crfm/BioMedLM")

model = GPT2LMHeadModel.from_pretrained("stanford-crfm/BioMedLM").to(device)

input_ids = tokenizer.encode(
    "Photosynthesis is ", return_tensors="pt"
).to(device)

sample_output = model.generate(input_ids, do_sample=True, max_length=50, top_k=50)

print("Output:\n" + 100 * "-")
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

c:\Users\facua\anaconda3\envs\chlorine\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\facua\anaconda3\envs\chlorine\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\facua\.cache\huggingface\hub\models--stanford-crfm--BioMedLM. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see ac

KeyboardInterrupt: 